# Проект: Игра-бот по серии книг о Гарри Поттере от Дж. Роулинг

## 1. Предобработка текста книг о Гарри Поттере Дж. Роулинг

Воспользуемся библиотекой `razdel` для того чтобы разделить наш текст на предложения. Выбор данной библиотеки обусловлен тем, что при пременении регулярных выражений не все предложения бились корректно. Несмотря на то, что библиотека также основана на примененнии регулярных выражений, с её помощью результат получается лучше за счет учета особенность разбиения текстов на русском языке.

https://github.com/natasha/razdel

In [2]:
with open('harry_potter.txt') as f:
    lines = f.read().splitlines()

In [3]:
!pip install razdel

In [4]:
import re
from razdel import sentenize

hp_sentences = []
for line in lines[428:]: # без аннотации, содержания и автора (первые 428 строк)
    if line != '' and line != '* * *':
        # result = re.split(r'(?<=\w[.!?]) ', line)
        # hp_sentences.extend(result)
        sentence = list(sentenize(line))
        hp_sentences.extend([_.text for _ in sentence])

In [5]:
hp_sentences[:10]

['Книга 1 - Гарри Поттер и Философский Камень',
 'Глава 1.',
 'Мальчик, который выжил',
 'Мистер и миссис Дурсль проживали в доме номер четыре по Тисовой улице и всегда с гордостью заявляли, что они, слава богу, абсолютно нормальные люди.',
 'Уж от кого-кого, а от них никак нельзя было ожидать, чтобы они попали в какую-нибудь странную или загадочную ситуацию.',
 'Мистер и миссис Дурсль весьма неодобрительно относились к любым странностям, загадкам и прочей ерунде.',
 'Мистер Дурсль возглавлял фирму под названием «Граннингс», которая специализировалась на производстве дрелей.',
 'Это был полный мужчина с очень пышными усами и очень короткой шеей.',
 'Что же касается миссис Дурсль, она была тощей блондинкой с шеей почти вдвое длиннее, чем положено при ее росте.',
 'Однако этот недостаток пришелся ей весьма кстати, поскольку большую часть времени миссис Дурсль следила за соседями и подслушивала их разговоры.']

Из полученных предложений начнем созадвать датасет, который пригодится нам на этапе тестирования моделей для распознавания искуственно сгенерированных предложений. Датасет будет иметь следующий вид: преложение и соотвествующий ему класс ('original' если это оригинальное предложение из текста, 'generated' - сгенерированное языковой моделью).


In [6]:
# из всех предложений возьмем более длинные
len_sent = 150 #30
hp_sentences_longer = []
for line in hp_sentences:
    if len(line) > len_sent:
        hp_sentences_longer.append(line)

In [7]:
hp_sentences_longer[:10]

['Когда во вторник мистер и миссис Дурсль проснулись скучным и серым утром - а именно с этого утра начинается наша история, - ничто, включая покрытое тучами небо, не предвещало, что вскоре по всей стране начнут происходить странные и загадочные вещи.',
 'А миссис Дурсль, с трудом усадив сопротивляющегося и орущего Дадли на высокий детский стульчик, со счастливой улыбкой пересказывала мужу последние сплетни.',
 'В половине девятого мистер Дурсль взял свой портфель, клюнул миссис Дурсль в щеку и попытался на прощанье поцеловать Дадли, но промахнулся, потому что Дадли впал в ярость, что с ним происходило довольно часто.',
 'Но когда он подъехал к Лондону, заполнившие его голову дрели вылетели оттуда в мгновение ока, потому что, попав в обычную утреннюю автомобильную пробку и от нечего делать глядя по сторонам, мистер Дурсль заметил, что на улицах появилось множество очень странно одетых людей.',
 'Мистер Дурсль пришел в ярость, увидев, что некоторые из них совсем не молоды, - подумать тол

In [8]:
len(hp_sentences_longer)

5534

In [9]:
import pandas as pd

df_hp = pd.DataFrame(data=hp_sentences_longer, columns=['sentence'])
df_hp['class'] = 'original'
df_hp.head()

,sentence,class
0,Когда во вторник мистер и миссис Дурсль просну...,original
1,"А миссис Дурсль, с трудом усадив сопротивляюще...",original
2,В половине девятого мистер Дурсль взял свой по...,original
3,"Но когда он подъехал к Лондону, заполнившие ег...",original
4,"Мистер Дурсль пришел в ярость, увидев, что нек...",original


In [10]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
russian_stopwords = stopwords.words('russian') + list(punctuation)
print(russian_stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [34]:
train_word2vec_data = []
for sentence in hp_sentences:
    tokenized_sentence = nltk.word_tokenize(sentence.lower(), language="russian")
    cleaned_tokenized_sentence = []
    for i in tokenized_sentence:
        if i not in russian_stopwords:
            cleaned_tokenized_sentence.append(i)
    train_word2vec_data.append(cleaned_tokenized_sentence)

## 2. Генерации новых предложений на основе исходного текста

Для генерации новых преложений заменим часть слов в исходном предложении на наиболее близкие, с использованием __модели Word2Vec__. Обучим модель и заменим 25% слов в предложении на синонимы.


In [35]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count = 1, vector_size = 100, window = 5)
w2v_model.build_vocab(train_word2vec_data)

In [36]:
import time
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0
        self.start_time = time.time()

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        epoch_time = time.time() - self.start_time
        print('Loss after epoch {}: {}, time for epoch {} seconds'.format(self.epoch, loss_now, epoch_time))
        self.epoch += 1
        self.start_time = time.time()


w2v_model.train(train_word2vec_data,
                total_examples=w2v_model.corpus_count, epochs=100, report_delay=1,
                compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 755538.0, time for epoch 1.2792997360229492 seconds
Loss after epoch 1: 717736.125, time for epoch 1.206202745437622 seconds
Loss after epoch 2: 582349.625, time for epoch 1.3469796180725098 seconds
Loss after epoch 3: 554070.0, time for epoch 2.167193651199341 seconds
Loss after epoch 4: 473281.75, time for epoch 1.4090816974639893 seconds
Loss after epoch 5: 426843.0, time for epoch 1.2117831707000732 seconds
Loss after epoch 6: 452620.25, time for epoch 1.2088196277618408 seconds
Loss after epoch 7: 409472.75, time for epoch 1.2316269874572754 seconds
Loss after epoch 8: 405219.0, time for epoch 1.2366855144500732 seconds
Loss after epoch 9: 336648.0, time for epoch 1.2005853652954102 seconds
Loss after epoch 10: 310662.0, time for epoch 1.2093214988708496 seconds
Loss after epoch 11: 329461.5, time for epoch 1.2037956714630127 seconds
Loss after epoch 12: 292879.5, time for epoch 1.748042345046997 seconds
Loss after epoch 13: 295245.0, time for epoch 1.938103914

(56813359, 59345000)

In [54]:
import random

generated_sentences = []
frac = 0.25 # процент слов, которые будут заменены
for sentence in hp_sentences_longer:
    tokenized_sentence = nltk.word_tokenize(sentence, language="russian")
    len_tok_sent = len(tokenized_sentence)
    random.seed(42)
    word_inds = sorted(random.sample(list(range(len(tokenized_sentence))), int(frac*len(tokenized_sentence))))
    word_to_change = [tokenized_sentence[word_inds[i]] for i in range(len(word_inds))]
    replacements = []

    for word in word_to_change:
        if word in w2v_model.wv.key_to_index.keys(): # получится заменить только если слово есть в словаре
            replacements.append(w2v_model.wv.most_similar(positive=[word], topn = 1)[0][0])
        else:
            replacements.append(word)

    for index in range(len(word_inds)):
        tokenized_sentence[word_inds[index]] = replacements[index]
    generated_sentences.append(" ".join(tokenized_sentence))

In [37]:
words_for_synonyms = ['во', 'Дурсль', 'проснулись', 'скучным', 'именно', 'с', 'утра', 'вскоре', 'странные']
for word in words_for_synonyms:
    if word in w2v_model.wv.key_to_index.keys():
        top_synonyms = w2v_model.wv.most_similar(positive=[word], topn = 1)[0][0]
    else:
        top_synonyms = word
    print(f"Closest words to {word} = {top_synonyms}")

Closest words to во = во
Closest words to Дурсль = Дурсль
Closest words to проснулись = смеши
Closest words to скучным = утомленным
Closest words to именно = знание
Closest words to с = с
Closest words to утра = часа
Closest words to вскоре = дочкой
Closest words to странные = изображенные


In [55]:
generated_sentences[:10]

['Когда во вторник мистер и аккуратнее Дурсль смеши утомленным и серым утром - а знание с этого часа начинается наша история , - ничто , включая покрытое покрытое небо , не предвещало , что дочкой по всей стране начнут происходить изображенные и загадочные вещи .',
 'А миссис Дурсль , с трудом усадив орущего и орущего Дадли на высокий детский стульчик , со счастливой улыбкой стульчик пересказывала последние сплетни .',
 'В девятого девятого мистер Дурсль взял защитный мешочек , клюнул миссис Дурсль в щеку и пытался на помахав поцеловать Дадли , но промахнулся , потому что Дадли впал в ярость , что с ним происходило довольно часто .',
 'Но когда он подъехал к Лондону , надевали его голову дрели вылетели оттуда в зеницу воцарился , потому что , попав в обычную утреннюю автомобильную пробку и от нечего делать глядя по сторонам , мистера Дурсль заметил , что на курили появилось множество очень странно одетых людей .',
 'Мистер Дурсль пришел в обрушившегося , увидев , что некоторые из них с

In [80]:
import re

normal_generated_sentences = []

for sentence in generated_sentences:
    space_punc_delete = re.sub(r'\s([?.:,!"](?:\s|$))', r'\1', sentence)
    normal_generated_sentences.append(space_punc_delete)

In [83]:
len(normal_generated_sentences)

5534

In [91]:
df_hp_gen = pd.DataFrame(data=normal_generated_sentences, columns=['sentence'])
df_hp_gen['class'] = 'generated'
df_hp_gen.head()

,sentence,class
0,Когда во вторник мистер и аккуратнее Дурсль см...,generated
1,"А миссис Дурсль, с трудом усадив орущего и ору...",generated
2,В девятого девятого мистер Дурсль взял защитны...,generated
3,"Но когда он подъехал к Лондону, надевали его г...",generated
4,"Мистер Дурсль пришел в обрушившегося, увидев, ...",generated


In [116]:
random.seed(42)
train_inds = random.sample(list(range(len(df_hp))), round(0.8*len(df_hp)))
test_inds = list(set(list(range(len(df_hp)))) - set(train_inds))

In [122]:
df_train = pd.concat([df_hp.take(train_inds), df_hp_gen.take(train_inds)], axis=0)
df_test = pd.concat([df_hp.take(test_inds), df_hp_gen.take(test_inds)], axis=0)

In [123]:
df_train

,sentence,class
5238,"Длинные черные волосы струились по спине, глаз...",original
912,"- Затем, тупица, что на Чемпионат в страну съе...",original
204,"К несчастью, выяснилось, что именно за этой дв...",original
2253,- Спровоцировал? - выкрикнула она и так хватил...,original
2006,За столиком по соседству с Гарри и Роном пару ...,original
...,...,...
2999,"- Спасибо! - заорал Гарри, поспешая Невилла в ...",generated
398,Гарри и Рон вернулась в Общую гостиную Гриффин...,generated
5167,"Именно такая, как говорил Ксенофилиус: позорна...",generated
1776,"Но золотая нить, разрыва он был блестела, блес...",generated


In [128]:
df_train.to_csv("hp_train.csv")

In [124]:
df_test

,sentence,class
0,Когда во вторник мистер и миссис Дурсль просну...,original
4096,"- Рон, из-за тебя снег пошел, - страдальческим...",original
2054,"Крэбб и Гойл, уже расхохотавшиеся при мысли о ...",original
8,"В общем, мистер Дурсль решил, что ему совсем н...",original
11,За ужином она охотно рассказала мистеру Дурслю...,original
...,...,...
2034,"- Уймись, Гермиона, с ними все в полном порядк...",generated
2038,"- Вы должны помнить, - собственноручного водян...",generated
4090,Ты окрестного в околдовывая лорда… Волан-де-Мо...,generated
2043,В легкой панике соскользнули горы домашней раб...,generated


In [129]:
df_test.to_csv("hp_test.csv")

## 3. Проверим как с угадыванием справляется Naive Bayes

In [127]:
import pandas as pd
import numpy as np
from numpy import random

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer


X_train = df_train['sentence']
X_test = df_test['sentence']
y_train = df_train['class']
y_test = df_test['class']

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=['original', 'generated']))

accuracy 0.7312556458897922
              precision    recall  f1-score   support

    original       0.90      0.52      0.66      1107
   generated       0.66      0.94      0.78      1107

    accuracy                           0.73      2214
   macro avg       0.78      0.73      0.72      2214
weighted avg       0.78      0.73      0.72      2214



In [132]:
df_test['model_class'] = y_pred

In [134]:
df_test.to_csv("with_pred_hp.csv")

In [ ]:
df_test